In [2]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.ne

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-16 03:39:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.56MB/s    in 0.2s    

2022-11-16 03:39:03 (4.56 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# create spark session

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,BooleanType,DateType
spark = SparkSession.builder.appName("big-data-challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
# Load file

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz" 
# get data from amazon S3 and  unzip extract files 

spark.sparkContext.addFile(url)

music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")

music_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....|2015-01-31 00:08:00|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|      

In [5]:

print ("The of records for the music data set is:",music_df.count())


The of records for the music data set is: 1688884


In [6]:
# Drop null values, by default is any which means will drop any rows which has a NA
music_df = music_df.dropna(how='any')
music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....|2015-01-31 00:08:00|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|      

In [7]:
# clean data by dropping NA and get count of data

print("The count after dropping NA ", music_df.count())

The count after dropping NA  1688606


In [8]:
#drop duplicates 
music_df = music_df.dropDuplicates(["review_id","customer_id", "product_id"])

print("The count after dropping dropping Duplicates ", music_df.count())

The count after dropping dropping Duplicates  1688606


In [9]:
#create dataframe that corresponds to customers table  with counts of records per user
customer_info_df = music_df.groupby("customer_id").agg({"customer_id": "count"})
customer_info_df = customer_info_df.withColumnRenamed("count(customer_id)", "customer_count")
customer_info_df.show(10)
customer_info_df.dtypes

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   24109146|             4|
|   32798514|             2|
|   14726743|             6|
|     664644|             1|
|    5455445|             3|
|   36595320|             1|
|   37004417|             1|
|   14926470|             1|
|   46600029|           153|
|   44304979|             1|
+-----------+--------------+
only showing top 10 rows



[('customer_id', 'int'), ('customer_count', 'bigint')]

In [13]:
##create dataframe that corresponds to columns in Products to load into data
product_df = music_df.select(["product_id","product_title"])
product_df.show(10)
product_df.dtypes
print ("Count of Products", product_df.count())

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000YMOQZY|Knockin' On Heave...|
|B00ISAEC80|            Flawless|
|B00VBB6AVU|    Scandal of Grace|
|B008A5POJO|  I Surrender (Live)|
|B014CW2RN4|We Are Young (Jer...|
|B00DRA3EOY|        Lord, I Live|
|B009G3U0PW|  I'll Never Be Free|
|B013XBYQOS|    All of the Stars|
|B003L5H25W|Exile On Main Str...|
|B00136NVH4|     Bat Out Of Hell|
+----------+--------------------+
only showing top 10 rows

Count of Products 1688606


In [7]:
from pyspark.sql.functions import *
#create dataframe that corresponds to columns in vine_table to load into data
vine_df = music_df.select(["review_id",col("star_rating").cast('int').alias("star_rating"),"helpful_votes","total_votes","vine"])
vine_df.show(10)
vine_df.dtypes
print ("Number of Vines", vine_df.count())

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R14LVB34Z2Z53I|          5|            0|          0|   N|
|R2M0R9CWPC82LT|          5|            0|          0|   N|
| RQNQI2R3BM52A|          4|            0|          0|   N|
| RTUIX1135HUAB|          5|            0|          0|   N|
| RE3QSAT8TL010|          4|            0|          0|   N|
|R14TN65IY0NUOD|          5|            0|          0|   N|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows

Number of Vines 1688606


In [8]:

#create dataframe that corresponds to columns in vine_table to load into data
review_id_df = music_df.select(["review_id","customer_id","product_id","product_parent", 
                                 to_date(col("review_date"),"yyyy-mm-dd").alias("review_date")])
review_id_df.show(10)
review_id_df.dtypes
print("Number of Review Ids", review_id_df.count())

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R14LVB34Z2Z53I|   10293599|B000YMOQZY|     262935067| 2015-01-31|
|R2M0R9CWPC82LT|    6087195|B00ISAEC80|     452318038| 2015-01-31|
| RQNQI2R3BM52A|   33717063|B00VBB6AVU|     675969129| 2015-01-31|
| RTUIX1135HUAB|   14948020|B008A5POJO|     282925893| 2015-01-31|
| RE3QSAT8TL010|   21694522|B014CW2RN4|     109077793| 2015-01-31|
|R14TN65IY0NUOD|   14948020|B00DRA3EOY|     959245080| 2015-01-31|
|R3LIF8MCNH1ODF|   20031966|B009G3U0PW|     762436916| 2015-01-31|
|R33X0DGA4PPQ3L|   12223745|B013XBYQOS|     137980344| 2015-01-31|
|R2DJJP7DQRT1ZW|   30231876|B003L5H25W|     165995375| 2015-01-31|
|R2A8XDXW5XOT4H|   51714365|B00136NVH4|     227508666| 2015-01-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows

Number of Review Ids 1688606


In [10]:
# Configure settings to connect Postgres
mode = "append"  # appending data to table that already has data for ebooks
jdbc_url="jdbc:postgresql:****:5432/****"
config = {"user":"postgres", 
          "password": "*****", 
          "driver":"org.postgresql.Driver"}


In [14]:
#review the initial count of  how many records in the table
customer_tab_cnt = spark.read.jdbc(url=jdbc_url, table='customers', properties=config)
print("Initial Table count for customers table:", customer_tab_cnt.count())

Initial Table count for customers table: 4030694


In [15]:
print ("The number of records in the customers dataframe:", customer_info_df.count())

The number of records in the customers dataframe: 798772


In [16]:
#load data into customers table in AWS Postgres database
customer_info_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [17]:
customer_cnt_after = spark.read.jdbc(url=jdbc_url, table='customers', properties=config)
print("Table count after data is loaded into customers table:", customer_cnt_after.count())

Table count after data is loaded into customers table: 4829466


In [18]:
print ("The number of records in the review_id_table dataframe:", review_id_df.count())

The number of records in the review_id_table dataframe: 1688606


In [19]:
#review the initial count of  how many records in the table
review_id_tab_cnt = spark.read.jdbc(url=jdbc_url, table='review_id_table', properties=config)
print("Initial Table count for review_id_table :", review_id_tab_cnt.count())

Initial Table count for review_id_table : 12518702


In [20]:
 # load  data into review_id_table AWS Postgres database
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [11]:
review_id_cnt_after = spark.read.jdbc(url=jdbc_url, table='review_id_table', properties=config)
print("Table count after data loaded into review_id_table :", review_id_cnt_after.count())

Table count after data loaded into review_id_table : 14207308


In [14]:
print ("The number of records in the products dataframe:", product_df.count())

The number of records in the products dataframe: 1688606


In [15]:
#review the initial count of  how many records in the table

products_table_count = spark.read.jdbc(url=jdbc_url, table='products', properties=config)
print("Initial Table count for products:", products_table_count.count())

Initial Table count for products: 12518702


In [16]:
#load data into Products table in AWS Postgres database
#review how many records in the table 
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [17]:
products_cnt_after = spark.read.jdbc(url=jdbc_url, table='products',properties=config)
print("Table count after data loaded into products table:", products_cnt_after.count())

Table count after data loaded into products table: 14207308


In [18]:
print ("The number of records in the vines_df dataframe:", vine_df.count())

The number of records in the vines_df dataframe: 1688606


In [19]:
#review the initial count of  how many records in the table

vine_table_count = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
print("Initial Table count for vine_table:", vine_table_count.count())

Initial Table count for vine_table: 12518702


In [20]:
#load data into vine_table in AWS Postgres Database
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [21]:
vine_tab_cnt = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
print("Table count after data loaded into vine_table:", vine_tab_cnt.count())

Table count after data loaded into vine_table: 14207308
